In [1]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [3]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['03.jpg',
 '05.jpg',
 'image_2160.jpg',
 'image_2620.jpg',
 'model',
 '27.ipynb',
 '30.ipynb',
 'FaceImage',
 '31.ipynb',
 'done_data_face_features.pickle',
 '28.ipynb',
 '29.ipynb',
 '32.ipynb']

In [4]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/done_data_face_features.pickle", mode="rb"))

In [5]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.10126288  0.04939684 -0.0135333  ...  0.00050823  0.08076832
    0.02925757]]

 [[ 0.11484744  0.06094224  0.00180208 ... -0.01531954  0.12997243
    0.02931331]]

 [[ 0.0883371   0.07687377 -0.01829172 ...  0.03242559  0.09040125
    0.00474092]]

 ...

 [[ 0.10115854  0.26688126 -0.05408768 ... -0.08945148  0.16420038
    0.0283456 ]]

 [[ 0.1297804   0.09853813 -0.07152011 ... -0.02761066  0.08419115
   -0.00754967]]

 [[ 0.06435601  0.00615203 -0.04063413 ...  0.11231432  0.06116884
    0.05675695]]]
['1721031731_NguyenDongHung' '1721031731_NguyenDongHung'
 '1721031731_NguyenDongHung' ... '162000186_NguyenVanHuong'
 '162000186_NguyenVanHuong' '162000186_NguyenVanHuong']


In [6]:
x.shape

(4696, 1, 128)

In [7]:
y.shape

(4696,)

In [8]:
x = x.reshape(-1, 128)

In [9]:
x.shape

(4696, 128)

In [10]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [11]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4226, 128), (470, 128), (4226,), (470,))

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [13]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [14]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [15]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.73
Accurency Test = 0.70
F1 Score Train = 0.52
F1 Score Test = 0.52


In [16]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [17]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.92
Accurency Test = 0.84
F1 Score Train = 0.85
F1 Score Test = 0.75


In [18]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [19]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.63


In [20]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [21]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [22]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.97
Accurency Test = 0.82
F1 Score Train = 0.97
F1 Score Test = 0.74


In [23]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [24]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   9.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   6.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   9.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=  11.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=  14.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=  17.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=  10.2s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [26]:
model_best_estimator = model_grid.best_estimator_

In [27]:
model_grid.best_score_

0.8442984224788695

In [28]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))